In [7]:
# Import required libraries
import pandas as pd

# Load the datasets
urban_data = pd.read_csv('data/merged_urban_data.csv')
population_data = pd.read_csv('data/population_cleaned.csv')
hospital_data = pd.read_csv('data/hosbed_staff_merge.csv')

# Inspect first few rows of each dataset
print("Urban Data Sample:")
print(urban_data.head(), "\n")

print("Population Data Sample:")
print(population_data.head(), "\n")

print("Hospital Data Sample:")
print(hospital_data.head(), "\n")

# Optionally clean or filter datasets if needed (example: only year 2020 for consistency)
urban_2020 = urban_data[urban_data['year'] == 2020]
population_2020 = population_data[population_data['year'] >= 2020]
hospital_2020 = hospital_data[hospital_data['year'] >= 2020]

# Aggregating population by state for 2020
state_population = population_2020.groupby('state')['population'].sum().reset_index()

# Merge hospital and population data on state
merged_hp = pd.merge(hospital_2020[hospital_2020['staff_type'] == 'all'], state_population, on='state', how='left')

# Now merge with urban data if needed (you can join on 'state' if present, or just keep separate)
# Note: urban_data is national-level, so merging by 'state' might not be applicable directly

# Save merged file for further use
merged_hp.to_csv('data/final_merged_data2025.csv', index=False)

print("✅ Merged data saved as 'final_merged_data2025.csv'")


Urban Data Sample:
  country_name country_iso3  year  \
0     Malaysia          MYS  1960   
1     Malaysia          MYS  1960   
2     Malaysia          MYS  1960   
3     Malaysia          MYS  1960   
4     Malaysia          MYS  1960   

                                      indicator_name     indicator_code  \
0           Urban population (% of total population)  SP.URB.TOTL.IN.ZS   
1  Population in urban agglomerations of more tha...  EN.URB.MCTY.TL.ZS   
2                         Population in largest city        EN.URB.LCTY   
3  Population in the largest city (% of urban pop...  EN.URB.LCTY.UR.ZS   
4                                   Urban population        SP.URB.TOTL   

          value state strata_x  percentage  urban_population strata_y  \
0  2.659800e+01   NaN      NaN         NaN         2116189.0      NaN   
1  4.317729e+00   NaN      NaN         NaN         2116189.0      NaN   
2  3.435270e+05   NaN      NaN         NaN         2116189.0      NaN   
3  1.623329e+01

In [3]:
import pandas as pd

# Load datasets
urban_data = pd.read_csv('data/merged_urban_data.csv')
population_data = pd.read_csv('data/population_cleaned.csv')
hospital_data = pd.read_csv('data/hosbed_staff_merge.csv')

# Define urban states manually
urban_states = ['Kuala Lumpur', 'Selangor', 'Penang', 'Johor', 'Melaka', 'Putrajaya']

# Function to classify area as Urban or Non-Urban
def classify_area(state):
    if state in urban_states:
        return 'Urban'
    else:
        return 'Non-Urban'

# Filter data for year 2020
population_2020 = population_data[population_data['year'] == 2020]
hospital_2020 = hospital_data[(hospital_data['year'] == 2020) & (hospital_data['staff_type'] == 'all')]

# Aggregate population by state
population_state_sum = population_2020.groupby('state')['population'].sum().reset_index()

# Apply strata classification to population and hospital data
population_state_sum['strata'] = population_state_sum['state'].apply(classify_area)
hospital_2020['strata'] = hospital_2020['state'].apply(classify_area)

# Merge hospital and population data on state and strata
merged_final = pd.merge(hospital_2020, population_state_sum, on=['state', 'strata'], how='left')

# Save to CSV
merged_final.to_csv('data/final_merged_with_strata.csv', index=False)

print(" Done! Saved as 'data/final_merged_with_strata.csv'")


 Done! Saved as 'data/final_merged_with_strata.csv'


C:\Users\Admin\AppData\Local\Temp\ipykernel_25464\3839943661.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hospital_2020['strata'] = hospital_2020['state'].apply(classify_area)


In [5]:
import pandas as pd

# Step 1: Load datasets
urban_data = pd.read_csv('data/merged_urban_data.csv')
population_data = pd.read_csv('data/population_cleaned.csv')
hospital_data = pd.read_csv('data/hosbed_staff_merge.csv')

# Step 2: Filter for year 2020
pop2020 = population_data[population_data['year'] >= 2020]
hos2020 = hospital_data[(hospital_data['year'] >= 2020) & (hospital_data['staff_type'] == 'all')]

# Step 3: Define Urban/Non-Urban states
urban_states = ['Kuala Lumpur', 'Selangor', 'Penang', 'Johor', 'Melaka', 'Putrajaya']
pop2020['strata'] = pop2020['state'].apply(lambda x: 'Urban' if x in urban_states else 'Non-Urban')
hos2020['strata'] = hos2020['state'].apply(lambda x: 'Urban' if x in urban_states else 'Non-Urban')

# Step 4: Reduce hospital data
hos2020_reduced = hos2020[['state', 'strata', 'beds', 'staff_count']].drop_duplicates()

# Step 5: Merge population with hospital data
expanded_data = pd.merge(pop2020, hos2020_reduced, on=['state', 'strata'], how='left')

# Step 6: Clean and pivot urban_data
urban_filtered = urban_data[
    urban_data['indicator_name'].isin([
        'Urban population (% of total population)',
        'Urban population'
    ])
]

# Aggregate to avoid duplicates
urban_grouped = urban_filtered.groupby(['year', 'indicator_name'])['value'].mean().unstack().reset_index()

# Get only 2020 data
urban_2020 = urban_grouped[urban_grouped['year'] >= 2020]

# Add urban indicators to every row of expanded_data
for col in urban_2020.columns:
    if col != 'year':
        expanded_data[col] = urban_2020[col].values[0]

# Step 7: Save final expanded dataset
expanded_data.to_csv('data/final_datausage.csv', index=False)

print(" Final dataset saved with urban indicators included.")


C:\Users\Admin\AppData\Local\Temp\ipykernel_25464\1250722123.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pop2020['strata'] = pop2020['state'].apply(lambda x: 'Urban' if x in urban_states else 'Non-Urban')
C:\Users\Admin\AppData\Local\Temp\ipykernel_25464\1250722123.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hos2020['strata'] = hos2020['state'].apply(lambda x: 'Urban' if x in urban_states else 'Non-Urban')


 Final dataset saved with urban indicators included.


In [9]:
import pandas as pd

# Step 1: Load datasets
urban_data = pd.read_csv('data/merged_urban_data.csv')
population_data = pd.read_csv('data/population_cleaned.csv')
hospital_data = pd.read_csv('data/hosbed_staff_merge.csv')

# Step 2: Filter for year >= 2020
pop_filtered = population_data[population_data['year'] >= 2020]
hos_filtered = hospital_data[(hospital_data['year'] >= 2020) & (hospital_data['staff_type'] == 'all')]

# Step 3: Define Urban/Non-Urban states
urban_states = ['Kuala Lumpur', 'Selangor', 'Penang', 'Johor', 'Melaka', 'Putrajaya']
pop_filtered['strata'] = pop_filtered['state'].apply(lambda x: 'Urban' if x in urban_states else 'Non-Urban')
hos_filtered['strata'] = hos_filtered['state'].apply(lambda x: 'Urban' if x in urban_states else 'Non-Urban')

# Step 4: Reduce hospital data
hos_reduced = hos_filtered[['year', 'state', 'strata', 'beds', 'staff_count']].drop_duplicates()

# Step 5: Merge population with hospital data (on year as well)
expanded_data = pd.merge(pop_filtered, hos_reduced, on=['year', 'state', 'strata'], how='left')

# Step 6: Clean and pivot urban_data
urban_filtered = urban_data[
    urban_data['indicator_name'].isin([
        'Urban population (% of total population)',
        'Urban population'
    ])
]

# Aggregate to avoid duplicates
urban_grouped = urban_filtered.groupby(['year', 'indicator_name'])['value'].mean().unstack().reset_index()

# Merge urban indicators into expanded_data by year
final_data = pd.merge(expanded_data, urban_grouped, on='year', how='left')

# Step 7: Save final expanded dataset
final_data.to_csv('data/final_datausage2025.csv', index=False)

print("Final dataset saved with urban indicators for all years from 2020 onward.")


C:\Users\Admin\AppData\Local\Temp\ipykernel_25464\3595736240.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hos_filtered['strata'] = hos_filtered['state'].apply(lambda x: 'Urban' if x in urban_states else 'Non-Urban')


Final dataset saved with urban indicators for all years from 2020 onward.
